# Test conversion on Niel's alpha-cyldextrin bound to 1-butylamine[+1] file!
```
/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/a-bam-p/a00
```

In [91]:
%load_ext autoreload
%autoreload 2

import numpy as np
import subprocess as sp

from openeye.oechem import *
from openforcefield.typing.engines.smirnoff import *
from openforcefield.utils import mergeStructure
import parmed as pmd

from utils import create_pdb_with_conect, prune_conect
from utils import split_topology, create_host_guest_topology
from utils import create_host_mol2, convert_mol2_to_sybyl
from utils import load_mol2, check_unique_atom_names, load_pdb
from utils import extract_water_and_ions, create_water_and_ions_parameters
from utils import extract_dummy_atoms, create_dummy_atom_parameters
from utils import map_residues, map_atoms
from utils import copy_box_vectors
from utils import rewrite_restraints_file, rewrite_amber_input_file
from utils import color_restraints

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Create a standards-compliant PDB...

In [2]:
create_pdb_with_conect(solvated_pdb='original/a-bam-p/full.crds', 
                       amber_prmtop='original/a-bam-p/full.topo', 
                       output_pdb='generated/a-bam-p/full.pdb')
prune_conect(input_pdb='full.pdb', 
             output_pdb='full_conect.pdb', 
             path='generated/a-bam-p/')

PDB file written by cpptraj.
First water residue = 160
Found first water CONECT entry at line = 8322


# Split the PDB into components and generate a topology for each of them...

In [3]:
components = split_topology(file_name='generated/a-bam-p/full.pdb')
hg_topology = create_host_guest_topology(components, host_resname='MGO', guest_resname='BAM')

# Load each topology component (from `mol2`) into separate `OEMol`s...

In [4]:
host = load_mol2(filename='original/a-bam-p/MGO.mol2', 
                 name='MGO', 
                 add_tripos=True)
guest = load_mol2(filename='original/a-bam-p/bam.mol2', 
                  name='BAM', 
                  add_tripos=False)
check_unique_atom_names(host)
check_unique_atom_names(guest)
molecules = [host, guest]

21 atoms in structure, 21 unique atom names.
17 atoms in structure, 17 unique atom names.


# Polymerize the host molecule and make sure there are *unique* atom types for every atom in the host...

In [5]:
create_host_mol2('generated/a-bam-p/full.pdb', 'original/a-bam-p/full.topo', 'MGO', 'generated/a-bam-p/MGO.mol2')

MOL2 file written by cpptraj.


In [93]:
convert_mol2_to_sybyl('generated/a-bam-p/MGO.mol2', 'generated/a-bam-p/MGO-sybyl.mol2')

In [94]:
host = load_mol2(filename='generated/a-bam-p/MGO-sybyl.mol2', 
                 name='MGO', 
                 add_tripos=True)
guest = load_mol2(filename='original/a-bam-p/bam.mol2', 
                  name='BAM', 
                  add_tripos=False)
check_unique_atom_names(host)
check_unique_atom_names(guest)
molecules = [host, guest]

126 atoms in structure, 126 unique atom names.
17 atoms in structure, 17 unique atom names.


# Parameterize the host and guest with SMIRNOFF99Frosst...

In [95]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml') 
system = ff.createSystem(hg_topology.topology, molecules,
                         nonbondedCutoff=1.1*unit.nanometer, 
                         ewaldErrorTolerance=1e-4
                         )

# Convert the parameterized system in a ParmEd structure -- now with the SMIRNOFF99Frosst parameters...

In [96]:
hg_structure = pmd.openmm.topsystem.load_topology(hg_topology.topology, system, hg_topology.positions)

# If I check the parameters here, we can see that some of the bond distances are set to 4.0, so we know there is a problem at this point!

In [97]:
pmd.tools.printBonds(hg_structure)

             Atom 1              Atom 2       R eq   Frc Cnst   Distance     Energy
      1   C1 (   1)       2   H1 (   2)     1.0900   340.0000     1.1077     0.1067
      1   C1 (   1)       3   O1 (   3)     4.0000  1000.0000     1.4676  6412.9044
      1   C1 (   1)       4   C2 (   4)     1.5260   310.0000     1.5259     0.0000
      1   C1 (   1)      16   O5 (  16)     4.0000  1000.0000     1.4738  6381.4472
      3   O1 (   3)      33   C4 (  33)     4.0000  1000.0000     1.4655  6423.7720
      4   C2 (   4)       5   H2 (   5)     1.0900   340.0000     1.0934     0.0040
      4   C2 (   4)       6   O2 (   6)     1.3700   320.0000     1.4372     1.4435
      4   C2 (   4)       8   C3 (   8)     1.5260   310.0000     1.5140     0.0446
      6   O2 (   6)       7  HO2 (   7)     4.0000  1000.0000     0.9698  9182.0059
      8   C3 (   8)       9   H3 (   9)     1.0900   340.0000     1.0930     0.0031
      8   C3 (   8)      10   O3 (  10)     1.3700   320.0000     1.4362    

# ...and save this as an AMBER-format `prmtop` and `inpcrd`...

In [9]:
try:
    hg_structure.save('generated/a-bam-p/hg.prmtop')
except OSError:
    print('Check if the file already exists...')

In [10]:
try:
    hg_structure.save('generated/a-bam-p/hg.inpcrd')
except OSError:
    print('Check if the file already exists...')

# Use the original AMBER files to extract the waters and ions (and dummy atoms) and then parameterize those with GAFF or something else...

In [11]:
extract_water_and_ions(amber_prmtop='original/a-bam-p/full.topo',
                      amber_inpcrd='original/a-bam-p/full.crds',
                      host_residue=':MGO',
                      guest_residue=':BAM',
                      dummy=None,
                      output_pdb='generated/a-bam-p/water_ions.pdb')

Water and ion PDB file written by cpptraj.


In [12]:
create_water_and_ions_parameters(input_pdb='water_ions.pdb',
                                output_prmtop='water_ions.prmtop',
                                output_inpcrd='water_ions.inpcrd',
                                dummy_atoms=True,
                                path='generated/a-bam-p/')

Writing dummy atom `frcmod`.
Writing dummy atom `mol2`.
Water and ion parameters and coordinates written by tleap.


# Load those parameters and coordinates into a ParmEd structure... and merge with the host-guest ParmEd structure...

In [13]:
water_and_ions = pmd.amber.AmberParm('generated/a-bam-p/water_ions.prmtop', xyz='generated/a-bam-p/water_ions.inpcrd')

In [14]:
merged = mergeStructure(hg_structure, water_and_ions)

In [15]:
try:
    merged.save('generated/a-bam-p/solvated_smirnoff.prmtop')
    merged.save('generated/a-bam-p/solvated_smirnoff.inpcrd')
except:
    print('Check if file exists...')

# Now, because we lumped the dummy atoms with the water and ions, they are no longer at the beginning of the coordinate file (if we extract the dummy atoms, then [ParmEd can't merge](https://github.com/ParmEd/ParmEd/issues/952) the structures). Thus, we need to remap all restraints to new atom and residue indices...

However, we need to do the *atom* mapping on a `mol2` file and the *residue* mapping on a `pdb` file -- where we can specifically choose to *not* ignore dummy atoms -- otherwise the graph is not isomorphic...

In [16]:
reference = pmd.load_file('original/a-bam-p/full.topo', 'original/a-bam-p/full.crds')
try:
    reference.save('generated/a-bam-p/reference.pdb')
    reference.save('generated/a-bam-p/reference.mol2')
except OSError:
    print('Check if file exists...')
target = pmd.load_file('generated/a-bam-p/solvated_smirnoff.prmtop', 'generated/a-bam-p/solvated_smirnoff.inpcrd')
try:
    target.save('generated/a-bam-p/target.pdb')
    target.save('generated/a-bam-p/target.mol2')
except OSError:
    print('Check if file exists...')

In [17]:
reference_mol = load_mol2('generated/a-bam-p/reference.mol2')
target_mol = load_mol2('generated/a-bam-p/target.mol2')

In [18]:
atom_mapping = map_atoms(reference_mol, target_mol)

Determining mapping...
Reference → Target
(C1      3 →   0 (C1   )
(H1      4 →   1 (H1   )
(Os1     5 →   2 (O1   )
(C2      6 →   3 (C2   )
(H2      7 →   4 (H2   )
(O1      8 →   5 (O2   )
(Ho1     9 →   6 (H3   )
(C3     10 →   7 (C3   )
(H3     11 →   8 (H4   )
(O2     12 →   9 (O3   )
(Ho2    13 →  10 (H5   )
(C4     14 →  11 (C4   )
(H4     15 →  12 (H6   )
(C5     16 →  13 (C5   )
(H5     17 →  14 (H7   )
(Os2    18 →  15 (O4   )
(C6     19 →  16 (C6   )
(H6     20 →  17 (H8   )
(H7     21 →  18 (H9   )
(O3     22 →  19 (O5   )
(Ho3    23 →  20 (H10  )
(C10    35 →  32 (C10  )
(C9     31 →  28 (C9   )
(C8     27 →  24 (C8   )
(C7     24 →  21 (C7   )
(H8     25 →  22 (H11  )
(Os3    26 →  23 (O6   )
(H9     28 →  25 (H12  )
(O4     29 →  26 (O7   )
(Ho4    30 →  27 (H13  )
(H10    32 →  29 (H14  )
(O5     33 →  30 (O8   )
(Ho5    34 →  31 (H15  )
(H11    36 →  33 (H16  )
(C11    37 →  34 (C11  )
(H12    38 →  35 (H17  )
(Os4    39 →  36 (O9   )
(C12    40 →  37 (C12  )
(H13    

(H936  1481 → 1481 (H954 )
(O460  1482 → 1482 (O472 )
(H937  1483 → 1483 (H955 )
(H938  1484 → 1484 (H956 )
(O461  1485 → 1485 (O473 )
(H939  1486 → 1486 (H957 )
(H940  1487 → 1487 (H958 )
(O462  1488 → 1488 (O474 )
(H941  1489 → 1489 (H959 )
(H942  1490 → 1490 (H960 )
(O463  1491 → 1491 (O475 )
(H943  1492 → 1492 (H961 )
(H944  1493 → 1493 (H962 )
(O464  1494 → 1494 (O476 )
(H945  1495 → 1495 (H963 )
(H946  1496 → 1496 (H964 )
(O465  1497 → 1497 (O477 )
(H947  1498 → 1498 (H965 )
(H948  1499 → 1499 (H966 )
(O466  1500 → 1500 (O478 )
(H949  1501 → 1501 (H967 )
(H950  1502 → 1502 (H968 )
(O467  1503 → 1503 (O479 )
(H951  1504 → 1504 (H969 )
(H952  1505 → 1505 (H970 )
(O468  1506 → 1506 (O480 )
(H953  1507 → 1507 (H971 )
(H954  1508 → 1508 (H972 )
(O469  1509 → 1509 (O481 )
(H955  1510 → 1510 (H973 )
(H956  1511 → 1511 (H974 )
(O470  1512 → 1512 (O482 )
(H957  1513 → 1513 (H975 )
(H958  1514 → 1514 (H976 )
(O471  1515 → 1515 (O483 )
(H959  1516 → 1516 (H977 )
(H960  1517 → 1517 (H978 )
(

(H1687 2608 → 2608 (H1705)
(H1688 2609 → 2609 (H1706)
(O836  2610 → 2610 (O848 )
(H1689 2611 → 2611 (H1707)
(H1690 2612 → 2612 (H1708)
(O837  2613 → 2613 (O849 )
(H1691 2614 → 2614 (H1709)
(H1692 2615 → 2615 (H1710)
(O838  2616 → 2616 (O850 )
(H1693 2617 → 2617 (H1711)
(H1694 2618 → 2618 (H1712)
(O839  2619 → 2619 (O851 )
(H1695 2620 → 2620 (H1713)
(H1696 2621 → 2621 (H1714)
(O840  2622 → 2622 (O852 )
(H1697 2623 → 2623 (H1715)
(H1698 2624 → 2624 (H1716)
(O841  2625 → 2625 (O853 )
(H1699 2626 → 2626 (H1717)
(H1700 2627 → 2627 (H1718)
(O842  2628 → 2628 (O854 )
(H1701 2629 → 2629 (H1719)
(H1702 2630 → 2630 (H1720)
(O843  2631 → 2631 (O855 )
(H1703 2632 → 2632 (H1721)
(H1704 2633 → 2633 (H1722)
(O844  2634 → 2634 (O856 )
(H1705 2635 → 2635 (H1723)
(H1706 2636 → 2636 (H1724)
(O845  2637 → 2637 (O857 )
(H1707 2638 → 2638 (H1725)
(H1708 2639 → 2639 (H1726)
(O846  2640 → 2640 (O858 )
(H1709 2641 → 2641 (H1727)
(H1710 2642 → 2642 (H1728)
(O847  2643 → 2643 (O859 )
(H1711 2644 → 2644 (H1729)
(

(O1160 3582 → 3582 (O1172)
(H2337 3583 → 3583 (H2355)
(H2338 3584 → 3584 (H2356)
(O1161 3585 → 3585 (O1173)
(H2339 3586 → 3586 (H2357)
(H2340 3587 → 3587 (H2358)
(O1162 3588 → 3588 (O1174)
(H2341 3589 → 3589 (H2359)
(H2342 3590 → 3590 (H2360)
(O1163 3591 → 3591 (O1175)
(H2343 3592 → 3592 (H2361)
(H2344 3593 → 3593 (H2362)
(O1164 3594 → 3594 (O1176)
(H2345 3595 → 3595 (H2363)
(H2346 3596 → 3596 (H2364)
(O1165 3597 → 3597 (O1177)
(H2347 3598 → 3598 (H2365)
(H2348 3599 → 3599 (H2366)
(O1166 3600 → 3600 (O1178)
(H2349 3601 → 3601 (H2367)
(H2350 3602 → 3602 (H2368)
(O1167 3603 → 3603 (O1179)
(H2351 3604 → 3604 (H2369)
(H2352 3605 → 3605 (H2370)
(O1168 3606 → 3606 (O1180)
(H2353 3607 → 3607 (H2371)
(H2354 3608 → 3608 (H2372)
(O1169 3609 → 3609 (O1181)
(H2355 3610 → 3610 (H2373)
(H2356 3611 → 3611 (H2374)
(O1170 3612 → 3612 (O1182)
(H2357 3613 → 3613 (H2375)
(H2358 3614 → 3614 (H2376)
(O1171 3615 → 3615 (O1183)
(H2359 3616 → 3616 (H2377)
(H2360 3617 → 3617 (H2378)
(O1172 3618 → 3618 (O1184)
(

(O1466 4500 → 4500 (O1478)
(H2949 4501 → 4501 (H2967)
(H2950 4502 → 4502 (H2968)
(O1467 4503 → 4503 (O1479)
(H2951 4504 → 4504 (H2969)
(H2952 4505 → 4505 (H2970)
(O1468 4506 → 4506 (O1480)
(H2953 4507 → 4507 (H2971)
(H2954 4508 → 4508 (H2972)
(O1469 4509 → 4509 (O1481)
(H2955 4510 → 4510 (H2973)
(H2956 4511 → 4511 (H2974)
(O1470 4512 → 4512 (O1482)
(H2957 4513 → 4513 (H2975)
(H2958 4514 → 4514 (H2976)
(O1471 4515 → 4515 (O1483)
(H2959 4516 → 4516 (H2977)
(H2960 4517 → 4517 (H2978)
(O1472 4518 → 4518 (O1484)
(H2961 4519 → 4519 (H2979)
(H2962 4520 → 4520 (H2980)
(O1473 4521 → 4521 (O1485)
(H2963 4522 → 4522 (H2981)
(H2964 4523 → 4523 (H2982)
(O1474 4524 → 4524 (O1486)
(H2965 4525 → 4525 (H2983)
(H2966 4526 → 4526 (H2984)
(O1475 4527 → 4527 (O1487)
(H2967 4528 → 4528 (H2985)
(H2968 4529 → 4529 (H2986)
(O1476 4530 → 4530 (O1488)
(H2969 4531 → 4531 (H2987)
(H2970 4532 → 4532 (H2988)
(O1477 4533 → 4533 (O1489)
(H2971 4534 → 4534 (H2989)
(H2972 4535 → 4535 (H2990)
(O1478 4536 → 4536 (O1490)
(

(H3530 5372 → 5372 (H3548)
(O1757 5373 → 5373 (O1769)
(H3531 5374 → 5374 (H3549)
(H3532 5375 → 5375 (H3550)
(O1758 5376 → 5376 (O1770)
(H3533 5377 → 5377 (H3551)
(H3534 5378 → 5378 (H3552)
(O1759 5379 → 5379 (O1771)
(H3535 5380 → 5380 (H3553)
(H3536 5381 → 5381 (H3554)
(O1760 5382 → 5382 (O1772)
(H3537 5383 → 5383 (H3555)
(H3538 5384 → 5384 (H3556)
(O1761 5385 → 5385 (O1773)
(H3539 5386 → 5386 (H3557)
(H3540 5387 → 5387 (H3558)
(O1762 5388 → 5388 (O1774)
(H3541 5389 → 5389 (H3559)
(H3542 5390 → 5390 (H3560)
(O1763 5391 → 5391 (O1775)
(H3543 5392 → 5392 (H3561)
(H3544 5393 → 5393 (H3562)
(O1764 5394 → 5394 (O1776)
(H3545 5395 → 5395 (H3563)
(H3546 5396 → 5396 (H3564)
(O1765 5397 → 5397 (O1777)
(H3547 5398 → 5398 (H3565)
(H3548 5399 → 5399 (H3566)
(O1766 5400 → 5400 (O1778)
(H3549 5401 → 5401 (H3567)
(H3550 5402 → 5402 (H3568)
(O1767 5403 → 5403 (O1779)
(H3551 5404 → 5404 (H3569)
(H3552 5405 → 5405 (H3570)
(O1768 5406 → 5406 (O1780)
(H3553 5407 → 5407 (H3571)
(H3554 5408 → 5408 (H3572)
(

In [21]:
reference_mol = load_pdb('generated/a-bam-p/reference.pdb')
target_mol = load_pdb('generated/a-bam-p/target.pdb')

In [22]:
residue_mapping = map_residues(atom_mapping, reference_mol, target_mol)

Reference → Target
C1    MGO   (   3)    4 →    1 (   0) C1    MGO  
H1    MGO   (   4)    4 →    1 (   1) H1    MGO  
O1    MGO   (   5)    4 →    1 (   2) O1    MGO  
C2    MGO   (   6)    4 →    1 (   3) C2    MGO  
H2    MGO   (   7)    4 →    1 (   4) H2    MGO  
O2    MGO   (   8)    4 →    1 (   5) O2    MGO  
H3    MGO   (   9)    4 →    1 (   6) H3    MGO  
C3    MGO   (  10)    4 →    1 (   7) C3    MGO  
H4    MGO   (  11)    4 →    1 (   8) H4    MGO  
O3    MGO   (  12)    4 →    1 (   9) O3    MGO  
H5    MGO   (  13)    4 →    1 (  10) H5    MGO  
C4    MGO   (  14)    4 →    1 (  11) C4    MGO  
H6    MGO   (  15)    4 →    1 (  12) H6    MGO  
C5    MGO   (  16)    4 →    1 (  13) C5    MGO  
H7    MGO   (  17)    4 →    1 (  14) H7    MGO  
O4    MGO   (  18)    4 →    1 (  15) O4    MGO  
C6    MGO   (  19)    4 →    1 (  16) C6    MGO  
H8    MGO   (  20)    4 →    1 (  17) H8    MGO  
H9    MGO   (  21)    4 →    1 (  18) H9    MGO  
O5    MGO   (  22)    4 →    1 

O403  WAT   (1275)  396 →  396 (1275) O403  WAT  
H817  WAT   (1276)  396 →  396 (1276) H817  WAT  
H818  WAT   (1277)  396 →  396 (1277) H818  WAT  
O404  WAT   (1278)  397 →  397 (1278) O404  WAT  
H819  WAT   (1279)  397 →  397 (1279) H819  WAT  
H820  WAT   (1280)  397 →  397 (1280) H820  WAT  
O405  WAT   (1281)  398 →  398 (1281) O405  WAT  
H821  WAT   (1282)  398 →  398 (1282) H821  WAT  
H822  WAT   (1283)  398 →  398 (1283) H822  WAT  
O406  WAT   (1284)  399 →  399 (1284) O406  WAT  
H823  WAT   (1285)  399 →  399 (1285) H823  WAT  
H824  WAT   (1286)  399 →  399 (1286) H824  WAT  
O407  WAT   (1287)  400 →  400 (1287) O407  WAT  
H825  WAT   (1288)  400 →  400 (1288) H825  WAT  
H826  WAT   (1289)  400 →  400 (1289) H826  WAT  
O408  WAT   (1290)  401 →  401 (1290) O408  WAT  
H827  WAT   (1291)  401 →  401 (1291) H827  WAT  
H828  WAT   (1292)  401 →  401 (1292) H828  WAT  
O409  WAT   (1293)  402 →  402 (1293) O409  WAT  
H829  WAT   (1294)  402 →  402 (1294) H829  WAT  


O710  WAT   (2196)  703 →  703 (2196) O710  WAT  
H1431 WAT   (2197)  703 →  703 (2197) H1431 WAT  
H1432 WAT   (2198)  703 →  703 (2198) H1432 WAT  
O711  WAT   (2199)  704 →  704 (2199) O711  WAT  
H1433 WAT   (2200)  704 →  704 (2200) H1433 WAT  
H1434 WAT   (2201)  704 →  704 (2201) H1434 WAT  
O712  WAT   (2202)  705 →  705 (2202) O712  WAT  
H1435 WAT   (2203)  705 →  705 (2203) H1435 WAT  
H1436 WAT   (2204)  705 →  705 (2204) H1436 WAT  
O713  WAT   (2205)  706 →  706 (2205) O713  WAT  
H1437 WAT   (2206)  706 →  706 (2206) H1437 WAT  
H1438 WAT   (2207)  706 →  706 (2207) H1438 WAT  
O714  WAT   (2208)  707 →  707 (2208) O714  WAT  
H1439 WAT   (2209)  707 →  707 (2209) H1439 WAT  
H1440 WAT   (2210)  707 →  707 (2210) H1440 WAT  
O715  WAT   (2211)  708 →  708 (2211) O715  WAT  
H1441 WAT   (2212)  708 →  708 (2212) H1441 WAT  
H1442 WAT   (2213)  708 →  708 (2213) H1442 WAT  
O716  WAT   (2214)  709 →  709 (2214) O716  WAT  
H1443 WAT   (2215)  709 →  709 (2215) H1443 WAT  


O1003 WAT   (3075)  996 →  996 (3075) O1003 WAT  
H2017 WAT   (3076)  996 →  996 (3076) H2017 WAT  
H2018 WAT   (3077)  996 →  996 (3077) H2018 WAT  
O1004 WAT   (3078)  997 →  997 (3078) O1004 WAT  
H2019 WAT   (3079)  997 →  997 (3079) H2019 WAT  
H2020 WAT   (3080)  997 →  997 (3080) H2020 WAT  
O1005 WAT   (3081)  998 →  998 (3081) O1005 WAT  
H2021 WAT   (3082)  998 →  998 (3082) H2021 WAT  
H2022 WAT   (3083)  998 →  998 (3083) H2022 WAT  
O1006 WAT   (3084)  999 →  999 (3084) O1006 WAT  
H2023 WAT   (3085)  999 →  999 (3085) H2023 WAT  
H2024 WAT   (3086)  999 →  999 (3086) H2024 WAT  
O1007 WAT   (3087) 1000 → 1000 (3087) O1007 WAT  
H2025 WAT   (3088) 1000 → 1000 (3088) H2025 WAT  
H2026 WAT   (3089) 1000 → 1000 (3089) H2026 WAT  
O1008 WAT   (3090) 1001 → 1001 (3090) O1008 WAT  
H2027 WAT   (3091) 1001 → 1001 (3091) H2027 WAT  
H2028 WAT   (3092) 1001 → 1001 (3092) H2028 WAT  
O1009 WAT   (3093) 1002 → 1002 (3093) O1009 WAT  
H2029 WAT   (3094) 1002 → 1002 (3094) H2029 WAT  


H2575 WAT   (3913) 1275 → 1275 (3913) H2575 WAT  
H2576 WAT   (3914) 1275 → 1275 (3914) H2576 WAT  
O1283 WAT   (3915) 1276 → 1276 (3915) O1283 WAT  
H2577 WAT   (3916) 1276 → 1276 (3916) H2577 WAT  
H2578 WAT   (3917) 1276 → 1276 (3917) H2578 WAT  
O1284 WAT   (3918) 1277 → 1277 (3918) O1284 WAT  
H2579 WAT   (3919) 1277 → 1277 (3919) H2579 WAT  
H2580 WAT   (3920) 1277 → 1277 (3920) H2580 WAT  
O1285 WAT   (3921) 1278 → 1278 (3921) O1285 WAT  
H2581 WAT   (3922) 1278 → 1278 (3922) H2581 WAT  
H2582 WAT   (3923) 1278 → 1278 (3923) H2582 WAT  
O1286 WAT   (3924) 1279 → 1279 (3924) O1286 WAT  
H2583 WAT   (3925) 1279 → 1279 (3925) H2583 WAT  
H2584 WAT   (3926) 1279 → 1279 (3926) H2584 WAT  
O1287 WAT   (3927) 1280 → 1280 (3927) O1287 WAT  
H2585 WAT   (3928) 1280 → 1280 (3928) H2585 WAT  
H2586 WAT   (3929) 1280 → 1280 (3929) H2586 WAT  
O1288 WAT   (3930) 1281 → 1281 (3930) O1288 WAT  
H2587 WAT   (3931) 1281 → 1281 (3931) H2587 WAT  
H2588 WAT   (3932) 1281 → 1281 (3932) H2588 WAT  


H3209 WAT   (4864) 1592 → 1592 (4864) H3209 WAT  
H3210 WAT   (4865) 1592 → 1592 (4865) H3210 WAT  
O1600 WAT   (4866) 1593 → 1593 (4866) O1600 WAT  
H3211 WAT   (4867) 1593 → 1593 (4867) H3211 WAT  
H3212 WAT   (4868) 1593 → 1593 (4868) H3212 WAT  
O1601 WAT   (4869) 1594 → 1594 (4869) O1601 WAT  
H3213 WAT   (4870) 1594 → 1594 (4870) H3213 WAT  
H3214 WAT   (4871) 1594 → 1594 (4871) H3214 WAT  
O1602 WAT   (4872) 1595 → 1595 (4872) O1602 WAT  
H3215 WAT   (4873) 1595 → 1595 (4873) H3215 WAT  
H3216 WAT   (4874) 1595 → 1595 (4874) H3216 WAT  
O1603 WAT   (4875) 1596 → 1596 (4875) O1603 WAT  
H3217 WAT   (4876) 1596 → 1596 (4876) H3217 WAT  
H3218 WAT   (4877) 1596 → 1596 (4877) H3218 WAT  
O1604 WAT   (4878) 1597 → 1597 (4878) O1604 WAT  
H3219 WAT   (4879) 1597 → 1597 (4879) H3219 WAT  
H3220 WAT   (4880) 1597 → 1597 (4880) H3220 WAT  
O1605 WAT   (4881) 1598 → 1598 (4881) O1605 WAT  
H3221 WAT   (4882) 1598 → 1598 (4882) H3221 WAT  
H3222 WAT   (4883) 1598 → 1598 (4883) H3222 WAT  


H3811 WAT   (5767) 1893 → 1893 (5767) H3811 WAT  
H3812 WAT   (5768) 1893 → 1893 (5768) H3812 WAT  
O1901 WAT   (5769) 1894 → 1894 (5769) O1901 WAT  
H3813 WAT   (5770) 1894 → 1894 (5770) H3813 WAT  
H3814 WAT   (5771) 1894 → 1894 (5771) H3814 WAT  
O1902 WAT   (5772) 1895 → 1895 (5772) O1902 WAT  
H3815 WAT   (5773) 1895 → 1895 (5773) H3815 WAT  
H3816 WAT   (5774) 1895 → 1895 (5774) H3816 WAT  
O1903 WAT   (5775) 1896 → 1896 (5775) O1903 WAT  
H3817 WAT   (5776) 1896 → 1896 (5776) H3817 WAT  
H3818 WAT   (5777) 1896 → 1896 (5777) H3818 WAT  
O1904 WAT   (5778) 1897 → 1897 (5778) O1904 WAT  
H3819 WAT   (5779) 1897 → 1897 (5779) H3819 WAT  
H3820 WAT   (5780) 1897 → 1897 (5780) H3820 WAT  
O1905 WAT   (5781) 1898 → 1898 (5781) O1905 WAT  
H3821 WAT   (5782) 1898 → 1898 (5782) H3821 WAT  
H3822 WAT   (5783) 1898 → 1898 (5783) H3822 WAT  
O1906 WAT   (5784) 1899 → 1899 (5784) O1906 WAT  
H3823 WAT   (5785) 1899 → 1899 (5785) H3823 WAT  
H3824 WAT   (5786) 1899 → 1899 (5786) H3824 WAT  


# Now let's copy over the minimization input file, and replace the residue indices of the positional restraints...

In [66]:
rewrite_amber_input_file(reference_input='original/a-bam-p/therm1.in',
                        target_input='generated/a-bam-p/therm1.in',
                        reference_to_target_mapping=residue_mapping)

["':1-3", '|', ':10@C4', '|', ":10@N1',"] → ':8-10 | :7@C4 | :7@N1',


# Now let's copy over the restraint file, and replace the atom indices...

In [62]:
rewrite_restraints_file(reference_restraints='original/a-bam-p/disang.rest',
                       target_restraints='generated/a-bam-p/disang.rest',
                       reference_to_target_mapping=atom_mapping)

1,13,              → 144,10            
2,1,13,            → 145,144,10        
3,2,1,13,          → 146,145,144,10    
1,13,46,           → 144,10,43         
2,1,13,46,         → 145,144,10,43     
1,13,46,104,       → 144,10,43,101     
1,143,             → 144,140           
2,1,143,           → 145,144,140       
1,143,130,         → 144,140,127       
19,4,6,36,         → 16,1,3,33         
40,25,27,57,       → 37,22,24,54       
61,46,48,78,       → 58,43,45,75       
82,67,69,99,       → 79,64,66,96       
103,88,90,120,     → 100,85,87,117     
124,109,111,15,    → 121,106,108,12    
4,6,36,38,         → 1,3,33,35         
25,27,57,59,       → 22,24,54,56       
46,48,78,80,       → 43,45,75,77       
67,69,99,101,      → 64,66,96,98       
88,90,120,122,     → 85,87,117,119     
109,111,15,17,     → 106,108,12,14     
9,143              → 6,140             
23,143             → 20,140            
30,143             → 27,140            
44,143             → 41,140            


# Finally, copy over the box dimensions and angles, and we should be good to go...

In [25]:
copy_box_vectors(input_inpcrd='original/a-bam-p/full.crds',
                output_inpcrd='generated/a-bam-p/solvated_smirnoff.inpcrd')

# Check restraints look okay in Chimera

In [64]:
color_restraints('original/a-bam-p/disang.rest', color='red',
                 suffix='niel', 
                 md_file='niel.txt',
                path='generated/a-bam-p/render-restraints/')

In [65]:
color_restraints('generated/a-bam-p/disang.rest', color='blue',
                 suffix='dave', 
                 md_file='dave.txt',
                path='generated/a-bam-p/render-restraints/')

Positional restraint comma possibly misplaced!